In [ ]:
import powerplantmatching as pm
from powerplantmatching.utils import (
    config_filter,
    convert_to_short_name,
    set_column_name,
)
from powerplantmatching.cleaning import clean_name
import pandas as pd

In [ ]:
config = pm.get_config()
config["GNPT"]["status"] = [
    "operating",
    "construction",
    "pre-construction",
    "announced",
]
config["target_columns"] += ["Status"]

In [ ]:
df = pm.data.GNPT(raw=True)

In [ ]:
RENAME_COLUMNS = {
    "Project Name": "Name",
    "Capacity (MW)": "Capacity",
    "Latitude": "lat",
    "Longitude": "lon",
    "Start Year": "DateIn",
    "Retirement Year": "DateOut",
    "Country/Area": "Country",
    "GEM unit ID": "projectID",
}

status_list = config["GNPT"]["status"]

df = df.rename(columns=RENAME_COLUMNS)
df = (
    df.pipe(clean_name)
    .pipe(set_column_name, "GNPT")
    .pipe(convert_to_short_name)
    .dropna(subset="Capacity")
    .assign(
        DateIn=df["DateIn"].apply(pd.to_numeric, errors="coerce"),
        DateOut=df["DateOut"].apply(pd.to_numeric, errors="coerce"),
        lat=df["lat"].apply(pd.to_numeric, errors="coerce"),
        lon=df["lon"].apply(pd.to_numeric, errors="coerce"),
    )
    .query("Status in @status_list")
    .pipe(lambda x: x[df.columns.intersection(config.get("target_columns"))])
    .assign(Fueltype="Nuclear")
    .assign(Technology="Steam Turbine")
    .assign(Set="PP")
    .pipe(config_filter, config)
    .query("Country != 'Ukraine'")
    .powerplant.convert_country_to_alpha2()
)

In [ ]:
df.query("Country == 'SE'")

In [ ]:
rebuild_i = df.query("DateIn <= 1990 or DateOut <= 2045 or Status == 'announced'").index
df.loc[rebuild_i, "Capacity"] = 0.0
df.loc[rebuild_i, "DateIn"] = 2045
# df.drop("Status", axis=1, inplace=True, errors="ignore")
df.fillna({"DateIn": 2045}, inplace=True)
df

In [ ]:
df.to_csv("../pypsa-eur/data/custom_powerplants.csv")

In [ ]:
import geopandas as gpd

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

regions_fn = "../pypsa-eur/resources/20240826-z1/regions_onshore_base_s_115.geojson"
regions = gpd.read_file(regions_fn).set_index("name")

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df.lon, df.lat),
    crs="EPSG:4326",
)

crs = ccrs.EqualEarth()
fig, ax = plt.subplots(figsize=(7, 7), subplot_kw={"projection": crs})
regions.to_crs(crs.proj4_init).plot(
    ax=ax, color="lightgrey", edgecolor="grey", linewidth=0.5
)
gdf.to_crs(crs.proj4_init).plot(
    ax=ax,
    column=gdf.Status,
    cmap="tab10",
    markersize=gdf.Capacity / 10,
    alpha=0.5,
    legend=True,
    legend_kwds={"title": "Status", "loc": "upper left"},
)

plt.tight_layout()

plt.savefig("nuclear.pdf")